In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from transformers import TrainingArguments, Trainer, DistilBertForSequenceClassification, DistilBertTokenizer, pipeline
from datasets import load_dataset
import evaluate
import torch

In [7]:
# Load the IMDB dataset
ds = load_dataset("stanfordnlp/imdb")

In [8]:
# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

/Library/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
# Define a preprocessing function
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

In [10]:
# Apply the preprocessing function to the dataset
tokenized_datasets = ds.map(preprocess_function, batched=True)

In [11]:
# Remove unnecessary columns
tokenized_datasets = tokenized_datasets.remove_columns(['text'])

In [12]:
# Rename the label column to "labels"
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [13]:
# Set the format of the dataset
tokenized_datasets.set_format('torch')

In [14]:
# Load additional metrics if needed
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [15]:
# Define a function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {**accuracy, **f1}

In [16]:
# Load the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Define the training arguments with optimized hyperparameters
repo_name = "your-repo-name"  # Replace with your repo name
training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=3e-5,               # Adjusted learning rate
    per_device_train_batch_size=32,   # Adjusted batch size for training
    per_device_eval_batch_size=64,    # Adjusted batch size for evaluation
    num_train_epochs=4,               # Adjusted number of epochs
    weight_decay=0.01,                # Added weight decay
    warmup_steps=500,                 # Added warmup steps
    push_to_hub=True,
)

In [18]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/Library/anaconda3/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [19]:
# Train the model
trainer.train()

Step,Training Loss
500,0.394400
1000,0.210200
1500,0.157900
2000,0.091000
2500,0.071100
3000,0.039200


TrainOutput(global_step=3128, training_loss=0.15530664346102255, metrics={'train_runtime': 85482.4917, 'train_samples_per_second': 1.17, 'train_steps_per_second': 0.037, 'total_flos': 1.32467398656e+16, 'train_loss': 0.15530664346102255, 'epoch': 4.0})

In [21]:
import torch

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = torch.tensor(logits)  # Convert logits to a PyTorch tensor
    predictions = torch.argmax(logits, dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy, "f1": f1}

In [22]:
# Evaluate the model
results = trainer.evaluate()
print(results)

TypeError: argmax(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [23]:
from huggingface_hub import notebook_login

notebook_login()

In [27]:
from transformers import pipeline

In [31]:
from transformers import TrainingArguments

repo_name = "Iwaves/LHL"  # set your repository name here
training_args = TrainingArguments(
   output_dir=repo_name,
   push_to_hub=True,
)

In [32]:
# Load the model and tokenizer for predictions
model_repo = "Iwaves/LHL"  
my_model = pipeline('sentiment-analysis', model=model_repo)

ValueError: Could not load model Iwaves/LHL with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>). See the original errors:

while loading with AutoModelForSequenceClassification, an error is thrown:
Traceback (most recent call last):
  File "/Library/anaconda3/lib/python3.11/site-packages/transformers/pipelines/base.py", line 278, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/anaconda3/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py", line 566, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/anaconda3/lib/python3.11/site-packages/transformers/modeling_utils.py", line 3453, in from_pretrained
    raise EnvironmentError(
OS

In [33]:
# Push the model to the Hugging Face Hub
trainer.push_to_hub()

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-669040ae-63e419e40beee5023bb67f6a;fd752aa4-cba2-4b22-9dad-46f048318bee)

Repository Not Found for url: https://huggingface.co/api/models/Iwaves/your-repo-name/preupload/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.
Note: Creating a commit assumes that the repo already exists on the Huggingface Hub. Please use `create_repo` if it's not the case.

In [ ]:
# Example text data
data = [
    "I love this movie, it was fantastic!",
    "The film was boring and too long.",
    "An excellent performance by the lead actor.",
    "Not my cup of tea, I didn't enjoy it."
]


In [ ]:
# Make predictions
preds = my_model(data)


In [ ]:
# Inspect the predictions
for text, pred in zip(data, preds):
    print(f'Text: {text}')
    print(f'Prediction: {pred}\n')